Preprocessing

In [24]:
import pandas as pd
import re

In [25]:
# Load dataset
file_path = 'Rawdata_1000.csv'
df = pd.read_csv(file_path)

# Data Cleaning
df = df[['Judul', 'Content']]
df.to_csv('step1_data_cleaning_result.csv', index=False)
print("hasil Step 1 Data Cleaning")
print(df.head())

hasil Step 1 Data Cleaning
                                               Judul  \
0  Wali Kota Malang Beri Dukungan dan Motivasi ke...   
1  Prabowo Beberkan Rencana Pengembangan Komodita...   
2  Curhat ke Rano Karno, Warga Condet: Selama Ini...   
3  Asas Praduga Tak Bersalah, Polri Belum Tetapka...   
4  Sidang Kasus Bentrok Amsterdam: 5 Orang Dihuku...   

                                             Content  
0  Baca berita dengan sedikit iklan,  klik di sin...  
1  Baca berita dengan sedikit iklan,  klik di sin...  
2   JAKARTA, KOMPAS.com - Warga Jalan Eretan 2, C...  
3   JAKARTA, KOMPAS.com - Bareskrim Polri belum m...  
4  Masih ingat bentrokan di Belanda pada November...  


In [26]:

def filtering_content(text):
    if not isinstance(text, str):
        return ""
    
    # 1. Standarisasi karakter (hapus non-breaking space)
    text = text.replace('\xa0', ' ')
    
    # 2. Hapus Iklan & Navigasi Spesifik
    junk_phrases = [
        "Baca berita dengan sedikit iklan,  klik di sini",
        "Baca berita dengan sedikit iklan, klik di sini",
        "Scroll ke bawah untuk melanjutkan membaca",
        "Ringkasan Berita",
        "INFO NASIONAL –",
        "INFO TEMPO –",
        "(*) "
    ]
    for phrase in junk_phrases:
        text = text.replace(phrase, " ")

    # 3. Hapus Header Media di awal teks (Lebih Fleksibel)
    # Menangani: "TEMPO.CO, Jakarta -", "Solo -", "JAKARTA, KOMPAS.com -"
    # Menggunakan \s* di depan untuk antisipasi spasi liar di awal
    text = re.sub(r'^\s*[A-Za-z0-9\.]+\s*,\s*[A-Za-z\s]+\s*[-–—]\s*', '', text) 
    text = re.sub(r'^\s*[A-Z\s]+,\s*[A-Za-z0-9\.]+\s*[-–—]\s*', '', text)
    text = re.sub(r'^\s*[A-Z][a-z]+\s*[-–—]\s*', '', text) # Contoh: Solo -
    text = re.sub(r'^\s*[A-Z\s]+\s*[-–—]\s*', '', text)    # Contoh: JAKARTA -

    # 4. Hapus Link Video & Pilihan Editor (beserta judul beritanya)
    # Menghapus frasa dan teks setelahnya sampai ketemu baris baru atau spasi ganda
    text = re.sub(r'(Simak|Lihat|Baca)\s+Juga\s+Video:?.*?(?=\n|\r|\s{2,}|$)', ' ', text, flags=re.IGNORECASE)
    text = re.sub(r'Pilihan\s+Editor:?.*?(?=\n|\r|\s{2,}|$)', ' ', text, flags=re.IGNORECASE)
    
    # 5. Hapus Kredit Metadata (Editor, Penulis, Foto, dll)
    text = re.sub(r'(Video|Editor|Sumber|Foto|Videographer|Penulis|Reporter):\s*.*?(?=\n|\r|\s{2,}|$)', ' ', text, flags=re.IGNORECASE)
    
    # 6. Hapus Atribusi Foto/Kontributor
    text = re.sub(r'Dok\.\s+[A-Za-z\s]+', ' ', text)
    text = re.sub(r'[A-Z\s|]+\sberkontribusi dalam penulisan artikel ini\.', ' ', text)

    # 7. Bersihkan spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Terapkan pembersihan
df['Content'] = df['Content'].apply(filtering_content)

# Simpan hasil
df.to_csv('step2_filtering_result.csv', index=False)
print("hasil Step 2 Data Filtering")
print(df['Content'].head())

hasil Step 2 Data Filtering
0    Wali Kota Malang, Wahyu Hidayat, memberikan du...
1    Presiden Prabowo Subianto mengungkapkan rencan...
2    Warga Jalan Eretan 2, Condet, Jakarta Timur, m...
3    Bareskrim Polri belum menetapkan tersangka dal...
4    Masih ingat bentrokan di Belanda pada November...
Name: Content, dtype: object


In [27]:
df['Merged'] = df['Judul'] + " " + df['Content']

df['Merged'].to_csv('step3_Merged_result.csv', index=False)
print("hasil Step 3 Data Filtering")
print(df['Merged'].head())

hasil Step 3 Data Filtering
0    Wali Kota Malang Beri Dukungan dan Motivasi ke...
1    Prabowo Beberkan Rencana Pengembangan Komodita...
2    Curhat ke Rano Karno, Warga Condet: Selama Ini...
3    Asas Praduga Tak Bersalah, Polri Belum Tetapka...
4    Sidang Kasus Bentrok Amsterdam: 5 Orang Dihuku...
Name: Merged, dtype: object
